In [1]:
from nltk.corpus import wordnet as wn
import nltk

In [4]:
wn.synset('matter.n.03').hypernyms()

[Synset('physical_entity.n.01')]

In [5]:
wn.synset('physical_entity.n.01').hypernyms()

[Synset('entity.n.01')]

In [6]:
wn.synset('entity.n.01').hypernyms()

[]

In [9]:
wn.synset('cocoa.n.01').hypernyms()[0].hypernyms()

[Synset('food.n.01'), Synset('liquid.n.01')]

In [11]:
wn.synset('substance.n.07').hypernyms()

[Synset('matter.n.03')]

In [14]:
wn.synset('substance.n.01').hypernyms()

[Synset('matter.n.03'), Synset('part.n.01')]

In [15]:
wn.synset('matter.n.03').hypernyms()

[Synset('physical_entity.n.01')]